In [ ]:
import ccxt
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
import talib

# Initialize the Binance client
binance = ccxt.binance()

In [ ]:
class BinanceDataFetcher:
    def __init__(self, symbol: str, timeframe: str = '1h', limit: int = 5000):
        self.symbol = symbol
        self.timeframe = timeframe
        self.limit = limit

    def fetch_ohlcv(self):
        ohlcv = binance.fetch_ohlcv(self.symbol, self.timeframe, limit=self.limit)
        return ohlcv

    def ohlcv_to_dataframe(self, ohlcv):
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)
        return df

    def fetch_and_process_data(self):
        ohlcv = self.fetch_ohlcv()
        df = self.ohlcv_to_dataframe(ohlcv)
        return df
    
fetcher = BinanceDataFetcher('BTC/USDT')
df = fetcher.fetch_and_process_data()

In [ ]:
def visualize_data(df):
    plt.figure(figsize=(15, 7))
    df['close'].plot(title=f"{df.index[0].strftime('%Y-%m-%d')} to {df.index[-1].strftime('%Y-%m-%d')} BTC/USDT Hourly Closing Prices")
    plt.ylabel('Price')
    plt.xlabel('Timestamp')
    plt.grid()
    plt.tight_layout()
    plt.show()

visualize_data(df)

**Preprocess the data for FB Prophet**

In [ ]:
def preprocess_for_prophet(df):
    prophet_data = df.reset_index()
    prophet_data = prophet_data.rename(columns={'timestamp': 'ds', 'close': 'y'})
    return prophet_data

prophet_df = preprocess_for_prophet(df)
prophet_df.dropna(inplace=True)

print(prophet_df.columns)

**Computing technical indicators using TA-Lib**

In [ ]:
def compute_technical_indicators(df):
    df['RSI'] = talib.RSI(df['y'].values, timeperiod=14)
    df['MFI'] = talib.MFI(df['high'].values, df['low'].values, df['y'].values, df['volume'].values, timeperiod=14)
    df['OBV'] = talib.OBV(df['y'].values, df['volume'].values)
    df['ATR'] = talib.ATR(df['high'].values, df['low'].values, df['close'].values, timeperiod=14)
    df['Bollinger_Upper'], df['Bollinger_Middle'], df['Bollinger_Lower'] = talib.BBANDS(df['y'].values, timeperiod=20)
    df['Marubozu'] = df['open'] - df['y']
    df['Doji'] = (df['high'] - df['low']) / (df['open'] - df['y']).replace({0: 1})
    return df

df = compute_technical_indicators(df)

df.dropna(inplace=True)

**Train with FB Prophet**

In [ ]:
def train_prophet_model(prophet_df):
    model = Prophet(daily_seasonality=True, yearly_seasonality=False, weekly_seasonality=True)
    model.add_seasonality(name='hourly', period=24, fourier_order=8)  # Add custom hourly seasonality
    
    # Adding extra regressors
    for column in ['RSI', 'MFI', 'OBV', 'ATR', 'Bollinger_Upper', 'Bollinger_Middle', 'Bollinger_Lower', 'Marubozu', 'Doji']:
        model.add_regressor(column)
        
    model.fit(prophet_df)
    return model

model = train_prophet_model(prophet_df)

**Forecasting**

In [ ]:
# Let's forecast the next 48 hours (2 days) for BTC/USDT:

def forecast(model, df, periods=48):
    future = model.make_future_dataframe(periods=periods, freq='H')
    future = pd.concat([df, future])  # Concatenate old dataframe to compute technical indicators correctly
    future = compute_technical_indicators(future)
    
    # Only use the future timestamps for prediction
    future = future[future['ds'] > df['ds'].iloc[-1]]
    
    forecast = model.predict(future)
    return forecast

forecast_df = forecast(model, prophet_df)

**Visualize the forecast**

In [ ]:
def visualize_forecast(model, forecast_df):
    fig = model.plot(forecast_df, figsize=(15, 7))
    plt.title("BTC/USDT Price Forecast for Next 48 Hours")
    plt.xlabel('Timestamp')
    plt.ylabel('Price')
    plt.grid()
    plt.show()

visualize_forecast(model, forecast_df)

**Extract Confidence Interval**

In [ ]:
def extract_confidence_interval(forecast_df):
    # Extracting relevant columns
    relevant_columns = ['ds', 'yhat', 'yhat_lower', 'yhat_upper']

    # Filtering only future dates
    future_confidence_intervals = forecast_df[forecast_df['ds'] > prophet_df['ds'].iloc[-1]][relevant_columns]

    # Calculating the width of the confidence interval for each prediction
    future_confidence_intervals['interval_width'] = future_confidence_intervals['yhat_upper'] - future_confidence_intervals['yhat_lower']

    return future_confidence_intervals

confidence_intervals = extract_confidence_interval(forecast_df)

# Display the extracted confidence intervals for the next 48 hours
print(confidence_intervals)

**Deriving a Confidence Score**

In [ ]:
def calculate_confidence_score(row):
    relative_width = row['interval_width'] / row['yhat']
    # Assuming a maximum relative width of 0.1 for 100% confidence
    confidence = max(0, (0.1 - relative_width) * 10)
    return min(100, max(0, confidence * 100))

confidence_intervals['confidence_score'] = confidence_intervals.apply(calculate_confidence_score, axis=1)

for index, row in confidence_intervals.iterrows():
    timestamp = row['ds']
    confidence_score = row['confidence_score']
    print(f"Timestamp: {timestamp}, Confidence Score: {confidence_score:.2f}%")